In [1]:
import typing
dataset: typing.List[typing.Tuple[str, str]] = []
with open("assets/text/mandarin_train_partial_clean.txt", "r", encoding="utf-8") as f:
	lines = f.readlines()
	for i in range(0, len(lines), 2):
		text = lines[i].strip().strip('\n')
		phome = lines[i+1].strip().strip('\n')
		dataset.append((text, phome))
print(f"Got dataset: {len(dataset)}")

Got dataset: 1205


In [3]:
modified_ds = dataset.copy()
for k, (text, phome) in enumerate(dataset):
	newphome = []
	phomes = phome.split(' ')
	skip_one = False
	unhandle = False
	for i, ch in enumerate(text):
		if skip_one:
			skip_one = False
			continue
		if ch == "。" or ch == '.': newphome.append('.')
		elif ch == '，': newphome.append(',')
		elif ch == '-' or ch == '—': newphome.append('.')
		elif ch == '“' or ch == '”': newphome.append('\"')
		elif ch == '？': newphome.append('?')
		elif ch == '！': newphome.append('!')
		elif ch == '：': newphome.append(':')
		elif ch == '、': newphome.append(',')
		elif ch == '；': newphome.append(',')
		elif ch == '…': newphome.append('.')
		else:
			if len(phomes) == 0:
				unhandle = True
				break
			p = phomes.pop(0)
			if len(p) >= 3 and p[-2] == 'r' and text[i+1] == '儿':
				newphome.append(p[:-2] + p[-1])
				newphome.append('er2')
				skip_one = True
			else:
				newphome.append(p)
	if len(phomes) == 0 and not unhandle:
		modified_ds[k] = (text, ' '.join(newphome))
	else:
		print(f"Unhandle: {k}\n== {text}\n== {phome}\nremain: {phomes}\n")

Unhandle: 414
== 在邹焰焰闲聊的话音中，他那些高官“哥们儿”能量都不可小觑。
== zai4 zou1 yan4 yan4 xian2 liao2 de hua4 yin1 zhong1 ta1 na4 xie1 gao1 guan1 ge1 menr neng2 liang4 dou1 bu4 ke2 xiao3 qu4
remain: []



In [4]:
modified_ds[414]

('在邹焰焰闲聊的话音中，他那些高官“哥们儿”能量都不可小觑。',
 'zai4 zou1 yan4 yan4 xian2 liao2 de hua4 yin1 zhong1 ta1 na4 xie1 gao1 guan1 ge1 menr neng2 liang4 dou1 bu4 ke2 xiao3 qu4')

In [5]:
modified_ds[414] = (modified_ds[414][0], 'zai4 zou1 yan4 yan4 xian2 liao2 de hua4 yin1 zhong1 , ta1 na4 xie1 gao1 guan1 " ge1 men2 er2 " neng2 liang4 dou1 bu4 ke2 xiao3 qu4 .')

In [6]:
modified_ds[:5]

[('躺在急救担架上的男子双目紧闭，头发散发出一股烧焦的味道。',
  'tang3 zai4 ji2 jiu4 dan1 jia4 shang4 de nan2 zi shuang1 mu4 jin3 bi4 , tou2 fa4 san4 fa1 chu1 yi4 gu3 shao1 jiao1 de wei4 dao4 .'),
 ('工业园区是承接产业转移、加速产业集聚、培育产业集群的主要载体。',
  'gong1 ye4 yuan2 qu1 shi4 cheng2 jie1 chan3 ye4 zhuan3 yi2 , jia1 su4 chan3 ye4 ji2 ju4 , pei2 yu4 chan3 ye4 ji2 qun2 de zhu3 yao4 zai4 ti3 .'),
 ('那一刻，我才真正的懂你，就像懂我现在的自己。',
  'na4 yi2 ke4 , wo3 cai2 zhen1 zheng4 de dong2 ni3 , jiu4 xiang4 dong3 wo3 xian4 zai4 de zi4 ji3 .'),
 ('由于列车长时间停靠，车厢内的空气越来越“闷”。',
  'you2 yu2 lie4 che1 zhang3 shi2 jian1 ting2 kao4 , che1 xiang1 nei4 de kong1 qi4 yue4 lai2 yue4 " men1 " .'),
 ('但如果按车队规模，一嗨数千辆车的量级绝对算不上最大。',
  'dan4 ru2 guo3 an4 che1 dui4 gui1 mo2 , yi1 hai1 shu4 qian1 liang4 che1 de liang4 ji2 jue2 dui4 suan4 bu2 shang4 zui4 da4 .')]

In [7]:
with open("assets/text/mandarin_train_full_clean.txt", "w", encoding="utf-8") as f:
	for (text, phome) in modified_ds:
		f.write(text + "\n" + phome + "\n")

In [ ]:
invalid_text = []
for i, (text, phome) in enumerate(dataset):
	normalized = text.replace('，','').replace('。','').replace('.','').replace('—','').replace("“",'').replace("”",'').replace('？','').replace('！','').replace('：','').replace('！','').replace('、','').replace('；','').replace('…','')
	phome_count = len(phome.split(' '))
	if len(normalized)!= phome_count:
		invalid_text.append((i, text, normalized, phome, len(normalized), phome_count))
invalid_text

[('我媳妇儿天天跟我在家，每次我想抽烟都硬拉着我，啪啪啪。',
  '我媳妇儿天天跟我在家每次我想抽烟都硬拉着我啪啪啪',
  'wo3 xi2 fur4 tian1 tian1 gen1 wo3 zai4 jia1 mei3 ci4 wo2 xiang3 chou1 yan1 dou1 ying4 la1 zhe wo3 pa1 pa1 pa1',
  24,
  23),
 ('大海深处，早已铆足了劲儿的钻头终于钻入了地层之中。',
  '大海深处早已铆足了劲儿的钻头终于钻入了地层之中',
  'da4 hai3 shen1 chu4 zao2 yi2 mao3 zu2 le jinr4 de zuan4 tou zhong1 yu2 zuan1 ru4 le di4 ceng2 zhi1 zhong1',
  23,
  22),
 ('甘当“绿叶”的巩汉林提前一天就来到南京对词儿走台。',
  '甘当绿叶的巩汉林提前一天就来到南京对词儿走台',
  'gan1 dang1 lv4 ye4 de gong3 han4 lin2 ti2 qian2 yi4 tian1 jiu4 lai2 dao4 nan2 jing1 dui4 cir2 zou3 tai2',
  22,
  21),
 ('在邹焰焰闲聊的话音中，他那些高官“哥们儿”能量都不可小觑。',
  '在邹焰焰闲聊的话音中他那些高官哥们儿能量都不可小觑',
  'zai4 zou1 yan4 yan4 xian2 liao2 de hua4 yin1 zhong1 ta1 na4 xie1 gao1 guan1 ge1 menr neng2 liang4 dou1 bu4 ke2 xiao3 qu4',
  25,
  24),
 ('原来她把睡房灯泡拆下来整个儿吞掉，只吃剩下了一根电线。',
  '原来她把睡房灯泡拆下来整个儿吞掉只吃剩下了一根电线',
  'yuan2 lai2 ta1 ba3 shui4 fang2 deng1 pao4 chai1 xia4 lai zheng3 ger4 tun1 diao4 zhi3 chi1 sheng4 xia4 le yi4 gen1 dian4 xian4',
  25,
  24),
 ('在日本听罗斯特洛波维奇指挥的音乐会，我曾亲身到过那块儿土

In [20]:
import jieba, pypinyin

In [21]:
cuts = list(jieba.cut("赵蕊蕊从小喜欢画画，和许多女孩儿一样，她也很喜欢漫画书。"))
for cut in cuts:
	x = pypinyin.lazy_pinyin(cut, style=pypinyin.Style.TONE3, tone_sandhi=True)
	print(x)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.442 seconds.
Prefix dict has been built successfully.


['zhao4', 'rui3', 'rui3']
['cong2', 'xiao3']
['xi3', 'huan1']
['hua4', 'hua4']
['，']
['he2']
['xu3', 'duo1']
['nv3', 'hai2', 'er2']
['yi1', 'yang4']
['，']
['ta1']
['ye3']
['hen3']
['xi3', 'huan1']
['man4', 'hua4', 'shu1']
['。']
